In [1]:
import pandas as pd
import re
from os import listdir
from os.path import isfile, join
import gc
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import logging
from constants import *

In [26]:
goodids = pd.read_pickle(join(ETL_PATH, 'dewiki_good_ids.pickle'))
pattern = re.compile(r'dewiki_n')
wikipath = join(SMPL_PATH, 'dewiki')
files = sorted([f for f in listdir(wikipath)
                if isfile(join(wikipath, f))])

def docs_to_lists(token_series):
    return tuple(token_series.tolist())

tagged_documents = []
for name in files[:]:
    gc.collect()
    corpus = name.split('.')[0]
    print(corpus, end=', ')
    
    f = join(wikipath, name)
    df = pd.read_pickle(f)
    df = df[df.hash.isin(goodids.index)]
    # additional fixes (should have been earlier in the pipeline)
    mask = df.token.isin(['[', ']', '<', '>', '/'])
    df.loc[mask, POS] = PUNCT
    # remove punctuation only for doc2vec
    df = df[df.POS != PUNCT]
    df = df.groupby([HASH], sort=False)[TOKEN].agg(docs_to_lists)
    # the conversion of the hash_id to str is necessary since gensim trys to allocate an array for ids of size 2^64 if int values are too big.
    tagged_documents += [TaggedDocument(doc, [str(doc_id)]) for doc_id, doc in df.iteritems()]

len(tagged_documents)

dewiki_new_0_49999__10000_simple, 

8812

In [28]:
#model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4, seed=42)
model = Doc2Vec(vector_size=300, window=15, min_count=20, sample=1e-5, workers=3, hs=0, dm=0, negative=5, dbow_words=1, dm_concat=0, epochs=2, seed=42)
model.build_vocab(tagged_documents)

In [29]:
%time model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 4min 48s, sys: 368 ms, total: 4min 48s
Wall time: 1min 36s
